# Redes Neuronales - Trabajo Practico N°2 - Rossman Store Sales

## Normalization and Encoding

In [1]:
from sys import path
from os.path import realpath
path.append(realpath("../")) # Makes it works everywhere
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn_pandas import DataFrameMapper

In [2]:
df_train = pd.read_feather('../processed_dataset/all_preprocessed_train.fth')
df_test = pd.read_feather('../processed_dataset/all_preprocessed_test.fth')
df_train.shape, df_test.shape

((1017209, 81), (41088, 80))

In [3]:
# Verificación de que se hicieron bien las cosas en testa también
set(df_train.columns) - set(df_test.columns)

{'Customers', 'Sales'}

## Categorical vars

In [4]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 
            'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 
            'CompetitionOpenSinceYear', 'Promo2SinceYear', 
            'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 
            'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 
            'SchoolHoliday_bw']


## Continuos Vars

In [5]:
contin_vars = ['CompetitionDistance', 
               'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 
               'Precipitationmm', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
               'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 
               'trend_DE', 'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 
               'Promo', 'SchoolHoliday', 'StateHoliday_bool']

# Normalización en continuas y LabelEncode en Categoricas

In [6]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

mapper_cat = DataFrameMapper(cat_maps)
_ = mapper_cat.fit(df_train)

# Assortment es la novena posición en cat_maps
N = 10
print(list(zip(df_train['Assortment'].values[:N], mapper_cat.transform(df_train)[:,9][:N])))
print(list(zip(df_train['Events'].values[:N], mapper_cat.transform(df_train)[:,15][:N])))
print(list(zip(df_train['Year'].values[:N], mapper_cat.transform(df_train)[:,2][:N])))

[('a', 0), ('a', 0), ('a', 0), ('c', 2), ('a', 0), ('a', 0), ('c', 2), ('a', 0), ('c', 2), ('a', 0)]
[('Fog', 0), ('Fog', 0), ('Fog', 0), ('Sunny', 20), ('Sunny', 20), ('Sunny', 20), ('Rain', 10), ('Rain', 10), ('Fog', 0), ('Sunny', 20)]
[(2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2)]


In [7]:
mapper_cont = DataFrameMapper(contin_maps)
_ = mapper_cont.fit(df_train)

In [8]:
N = 10
print(df_train['CompetitionDistance'].values[:N])
print(mapper_cont.transform(df_train)[:, 0][:N])

[ 1270.   570. 14130.   620. 29910.   310. 24000.  7520.  2030.  3160.]
[-0.51104292 -0.59341201  1.00219483 -0.5875285   2.85902934 -0.62400624
  2.16359893  0.22439533 -0.42161363 -0.28864639]


In [9]:
# La hacemos con todas ahora
df_train[cat_vars] = mapper_cat.transform(df_train)
df_test[cat_vars] = mapper_cat.transform(df_test)
df_train[contin_vars] = mapper_cont.transform(df_train)
df_test[contin_vars] = mapper_cont.transform(df_test)

# Final processing

In [10]:
df_sales = df_train[df_train.Sales!=0]
df_sales.shape

(844338, 81)

In [11]:
df_test.reset_index(inplace=True)
df_sales.reset_index(inplace=True)
df_sales.to_feather('../processed_dataset/train_normalized_data.fth')
df_test.to_feather('../processed_dataset/test_normalized_data.fth')

In [12]:
(df_test['Open'] == 0).sum()

5984

In [13]:
(df_sales['Open'] == 0).sum()

0

In [14]:
(df_train['CompetitionDistance'].isna()).sum()

0